<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/Prompt_engineering/Mistral7_b_prompt_Enginiering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.0 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off


In [2]:
# Initial list
input_list = [15, 32, 5, 13, 82, 7, 1]

# Identify odd numbers
odd_numbers = [number for number in input_list if number % 2 != 0]

# Find the sum of odd numbers
odd_numbers_sum = sum(odd_numbers)

# Check if sum is also an odd number
if odd_numbers_sum % 2 != 0:
    answer = True  # sum is odd
else:
    answer = False  # sum is not odd

# Output
print('Odd numbers are: ', odd_numbers)
print('The sum of odd numbers is: ', odd_numbers_sum)
print('Is the sum of odd numbers also an odd number? ', answer)

Odd numbers are:  [15, 5, 13, 7, 1]
The sum of odd numbers is:  41
Is the sum of odd numbers also an odd number?  True


In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16, load_in_8bit=True
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 2048
generation_config.temperature = 0.0001
generation_config.do_sample = True


In [5]:

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


In [6]:

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [7]:

text = "[INST] You are a helpful assistant with arithmethic and mathematical capabilities, Lets think step by step. The odd numbers in this list = 15, 32, 5, 13, 82, 7, 1 add up to an even number. What is that number?  [/INST]"

In [8]:
%%time
result = llm(text)

To solve this problem, we need to find the sum of the odd numbers in the list:

15 + 32 + 5 + 13 + 82 + 7 + 1 = 197

Since the sum of the odd numbers is an even number, we can conclude that the sum of the odd numbers in the list is 197.
CPU times: user 23.5 s, sys: 118 ms, total: 23.6 s
Wall time: 24.6 s


In [9]:

text = """[INST] Role: You are a helpful assistant with arithmethic and mathematical capabilities, Lets think step by step.
Context: Odd numbers are numbers that cannot be divided by two, and they always end in 1, 3, 5, 7, or 9.
user:  take the odd numbers of this list = [15, 32, 5, 13, 82, 7, 1]. The list contains odd and non-odd numbers. Up to an only the odd numbers, and tell me the number and answer yes or not if the sum of the of numbers it is another even number. Lets think step by step.
example: user: list = [4, 7, 2, 1].   sump up of odd numbers 7 + 1 = 8 , 8 % 2 = 0.
Answer: Answer sump up of odd numbers is 8 and 8 is not and odd number [/INST]"""

In [10]:
%%time
result = llm(text)

Sure, I can help you with that!

To find the odd numbers in the given list, we can use a loop to iterate through each element in the list and check if it is odd or not. We can use the modulo operator (%) to check if a number is odd or not. If the remainder of a number divided by 2 is not 0, then the number is odd.

Here's the code to find the odd numbers in the given list:
```python
def find_odd_numbers(lst):
   odd_numbers = []
   for num in lst:
       if num % 2 != 0:
           odd_numbers.append(num)
   return odd_numbers

# Example usage
lst = [15, 32, 5, 13, 82, 7, 1]
odd_numbers = find_odd_numbers(lst)
print(odd_numbers)
```
This will output:
```
[5, 7, 1]
```
Now that we have the odd numbers in the list, we can calculate the sum of these numbers using the `sum()` function in Python.

Here's the code to calculate the sum of the odd numbers:
```python
def sum_odd_numbers(lst):
   odd_numbers = find_odd_numbers(lst)
   sum_of_odd_numbers = sum(odd_numbers)
   return sum_of_odd_nu

In [11]:
text = """[INST] Role: You are a helpful assistant able to generate SQL code. Lets think step by step.
Context:
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]

Create a MySQL query for all students in the Computer Science Department.

answer?
 [/INST]"""

In [12]:
%%time
result = llm(text)

Yes, I can help you with that. Here's the SQL code to retrieve all students in the Computer Science Department:
```sql
SELECT s.StudentId, s.StudentName
FROM students s
JOIN departments d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';
```
This query uses a `JOIN` statement to combine the `students` and `departments` tables based on the `DepartmentId` column. The `WHERE` clause filters the results to only include rows where the `DepartmentName` is 'Computer Science'. The `SELECT` statement specifies the columns to retrieve, which in this case are `StudentId` and `StudentName`.
CPU times: user 43.8 s, sys: 57.6 ms, total: 43.8 s
Wall time: 43.7 s
